# Ejemplo de funcionalidad principal

## Requerimientos

### Importamos las otras librerías utilizadas en este ejemplo

In [1]:
from pathlib import Path

import pandas as pd
import spacy

### Y lo que usaremos de la librería [banrep][pypi_banrep]
[pypi_banrep]: https://pypi.org/project/banrep/

In [2]:
from banrep.corpus import MiCorpus
from banrep.diagnosticos import docs_topicos, topico_dominante, palabras_probables
from banrep.io import Textos, leer_palabras
from banrep.topicos import crear_ldas, calcular_coherencias
from banrep.utils import crear_directorio

### Necesitamos un modelo NLP para español de [spaCy][spacy_models]
[spacy_models]: https://spacy.io/models

In [3]:
nlp = spacy.load('es_core_news_md')

## Preprocesamiento de texto y filtros

### Textos a usar

Asume una carpeta en disco en la que hay archivos de texto. Si los textos que quiere utilizar están en archivos binarios como *.pdf*, *.docx*, etc., debe primero [extraer el texto][extraccion].

[extraccion]: https://munozbravo.github.io/banrep/uso_extraccion/

#### Uso de clase: `Textos`

Se usa para iterar los archivos de texto en disco.

Parámetro `chars` filtra aquellas líneas de texto de cada archivo cuya longitud sea inferior al valor especificado. *69* es valor arbitrario que permite filtrar la mayoría de títulos y subtítulos.

Parámetro `parrafos` permite definir si se considera cada párrafo como un documento separado.

Parámetro `aleatorio` sirve cuando se quiere iterar los archivos aleatoriamente.

In [4]:
dir_datos = Path('../datasets/')

datos = Textos(dir_datos, aleatorio=False, chars=69, parrafos=True)
print(datos)

29 archivos en directorio datasets.


### Filtros: palabras y tokens a ignorar

Generalmente se quiere ignorar palabras comunes a todos los textos, llamadas *stopwords*, por no aportar al entendimiento de los diferentes textos.

Se obtendrá información detallada de cada palabra gracias a [spaCy][web_spacy], lo que permite filtrar adicionalmente por criterios como [categoría gramatical][universal] (verbos, sustantivos, etc), si es algún tipo de [nombre propio][spacy_ents] (Juan, Colombia, Banco de la República), o si contiene caracteres que no hacen parte del alfabeto (números, monedas, etc). 

[web_spacy]: https://spacy.io/
[universal]: https://universaldependencies.org/es/index.html
[spacy_ents]: https://spacy.io/api/annotation#named-entities

#### Uso de función: `leer_palabras`

Permite leer categorías de palabras de un archivo excel. 
En este caso, una sola categoría (stopwords).

Parámetros `archivo` y `hoja` determinan el archivo excel en disco, y la hoja a usar.

Parámetro `col_grupo` es el nombre de una columna en la hoja excel que determina el grupo al que pertenecen las palabras.

Parámetro `col_palabras` es el nombre de una columna en la hoja excel que contiene las palabras.

In [5]:
pathstops = '~/Dropbox/datasets/wordlists/stopwords.xlsx'
palabras = leer_palabras(archivo=pathstops, 
                         hoja='banrep_es', 
                         col_grupo="type", 
                         col_palabras="word")

stops = palabras.get("stopword")

#### Ilustración para mayor claridad

La hoja de stopwords en mi archivo excel...

![](img/stopwords.png)


En este caso elimino de cualquier análisis posterior todas las *stopwords* del archivo excel, las categorías gramaticales que identifican números, puntuación y símbolos, y aquellas "palabras" o tokens que contengan caracteres que no hacen parte del alfabeto.

In [6]:
tags = ['NUM', 'PUNCT', 'SYM']

filtros = dict(stopwords=stops, postags=tags, entities=None, is_alpha=True)

## Dimensiones adicionales que se quieren medir

### Pertenencia a listas de palabras predefinidas

Muchas veces se quiere contabilizar cuantas palabras de cada documento hacen parte de listas de palabras predefinidas. Por ejemplo, puedo tener listas de palabras "positivas" y "negativas", y querer contar cuantas palabras de los textos que voy a analizar hacen parte de estas listas. Esto sirve, por ejemplo, para crear indicadores de sentimiento basados en el conteo de palabras que pertenecen a emociones "contrarias".

#### Uso de función: `leer_palabras`

La misma función usada para cargar *stopwords* nos sirve para leer un archivo excel que contiene otras categorías de palabras, en este caso palabras que denotan *mejora* y otras que denotan *deterioro*.

In [7]:
pathwl = '~/Dropbox/datasets/wordlists/sentimiento.xlsx'
wordlists = leer_palabras(pathwl, 'BANREP', col_grupo="type", col_palabras="word")

In [8]:
# Cuantas palabras en cada grupo...

for tipo in wordlists:
    print(f'{len(wordlists.get(tipo))} palabras en grupo {tipo}')

1899 palabras en grupo deterioro
359 palabras en grupo mejora


## Corpus y Modelos

### Crear el corpus

Un *corpus* es un conjunto de documentos, para el cual queremos obtener toda clase de estadísticas.

`MiCorpus` es la implementación de un corpus, el cual se inicializa con un modelo [spaCy][spacy_models] y un objeto `Textos`, y opcionalmente con los filtros especificados anteriormente, las listas de palabras que se quiere contar, y expresiones que se quiere encontrar. 

[spacy_models]: https://spacy.io/models

#### Uso de clase: `MiCorpus`

Se usa para inicializar el corpus. Es la estructura más importante de [banrep][pypi_banrep].

Parámetros `lang` y `datos` son el modelo spaCy y el objeto Textos respectivamente.

Parámetros `filtros` y `wordlists` ya explicados anteriormente.

Parámetro `corta` sirve como filtro adicional, ignorando frases de pocas palabras (en este ejemplo, frases con menos de 10 palabras).

[pypi_banrep]: https://pypi.org/project/banrep/

In [9]:
corpus = MiCorpus(nlp, datos=datos, filtros=filtros, corta=9, wordlists=wordlists)
print(corpus)

Corpus con 5628 docs y 3619 palabras únicas.


### Crear modelos LDA

Los modelos de tópicos se usan para encontrar "temáticas" subyacentes en los textos.

El parámetro básico a especificar en un modelo es el número de tópicos que se quiere considerar en el resultado.

Esta librería usa [Gensim][web_gensim] para la implementación del cálculo de los modelos. En su [documentación][gensim_tuts] encontrará todo lo necesario para correr este tipo de modelos y muchas técnicas adicionales no usadas en esta librería. [banrep][pypi_banrep] simplemente ofrece funciones para correr varios modelos LDA y seleccionar el mejor, todo basado en Gensim.

[web_gensim]: https://radimrehurek.com/gensim/models/ldamodel.html
[gensim_tuts]: https://radimrehurek.com/gensim/tutorial.html
[pypi_banrep]: https://pypi.org/project/banrep/

#### Uso de función: `crear_ldas`

Se usa para crear modelos lda para diferentes números de tópicos.

Parámetro `corpus` es una instancia de `MiCorpus`, y parámetro `numeros` es una lista de números para los cuales se quiere generar un modelo.

Parámetro `params` es un diccionario con parámetros que se usan en la [implementación LDA de Gensim][lda_gensim] (gensim.models.ldamodel.LdaModel).

[lda_gensim]: https://radimrehurek.com/gensim/models/ldamodel.html

In [10]:
n_topicos = (5, 10, 15)
params = dict(passes=5, alpha='auto', eta='auto', random_state=100)

modelos = [lda for lda in crear_ldas(corpus, n_topicos, params)]

#### Uso de función: `calcular_coherencias`

Los modelos de tópicos suelen ser evaluados usando una medida llamada *Coherence Score*. Esta medida sugiere qué tan "interpretables" son los modelos. Un mayor score es un modelo más "interpretable", y por lo tanto mejor.

Parámetros `modelos` y `corpus` son, respectivamente, lista de modelos LDA previamente generados y una instancia de corpus `MiCorpus`.

In [11]:
scores = [score for score in calcular_coherencias(modelos, corpus)]

mejor_score = max(scores)
cual = scores.index(mejor_score)
mejor_n = n_topicos[cual]

print(f"Modelo de {mejor_n} tópicos mejor Coherence Score: {mejor_score}")

Modelo de 10 tópicos mejor Coherence Score: 0.45963354326637934


### Almacenamiento de objetos generados

#### Uso de función: `crear_directorio`

El corpus y los modelos de tópicos generan estructuras que pueden ser útiles más adelante, y por lo tanto se quiere guardar a disco.

`crear_directorio` simplemente crea un directorio en disco, en el que se quiere guardar las estructuras generadas.

Se quiere usualmente almacenar modelos de "ngramas", el "diccionario" que contiene todas las palabras únicas existentes en el corpus, y el mejor modelo de tópicos opcionalmente con  su visualización.

In [12]:
# Crear directorio de salida
dir_salida = crear_directorio('topicos')

#Guardar modelos de bigramas y trigramas
corpus.ngrams.get('bigrams').save(str(dir_salida.joinpath('bigrams')))
corpus.ngrams.get('trigrams').save(str(dir_salida.joinpath('trigrams')))

# Guardar diccionario
corpus.id2word.save(str(dir_salida.joinpath('id2word')))

In [13]:
# Guardar mejor modelo de tópicos y su visualización

dirtopic = dir_salida.joinpath(f'{mejor_n:0>2}')
crear_directorio(dirtopic)

modelo = modelos[cual]
modelo.save(str(dirtopic.joinpath('topicos.lda')))

# En análisis de tópicos se suele usar PyLDAvis para visualizar resultados...
# Hay que importar la librería para eso.

import warnings

import pyLDAvis
import pyLDAvis.gensim

# Gráfica LDAvis de tópicos y sus palabras

bow = list(corpus)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    vis = pyLDAvis.gensim.prepare(modelo, bow, corpus.id2word, sort_topics=False)

pyLDAvis.save_html(vis, str(dirtopic.joinpath('topicos.html')))

## Estadísticas de Modelo de tópicos

### Distribución de probabilidad de tópicos en documentos

Usualmente se quiere ver la probabilidad de cada tópico asociada a cada documento. Se puede  pensar un documento como algo generado de una distribución de probabilidad de tópicos. 

La función `docs_topicos` muestra dicha distribución. Para cada documento (fila) muestra la  probabilidad de que hable de cada tópico (columna).

Basado en esta distribución, `topico_dominante` muestra qué topico es dominante en cuantos documentos del corpus.

#### Uso de función: `docs_topicos`

Parámetros `modelo` y `corpus` son, respectivamente, un modelo LDA y una instancia de corpus `MiCorpus`.

In [14]:
doctopics = docs_topicos(modelo, corpus)
doctopics.tail()

,0,1,2,3,4,5,6,7,8,9
005624,0.262847,0.010089,0.014023,NaN,0.010598,0.011491,0.013072,0.465592,NaN,0.193156
005625,0.952283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
005626,0.580739,NaN,0.107427,NaN,NaN,0.132913,NaN,NaN,NaN,0.165784
005627,0.498586,0.029520,0.044318,NaN,NaN,NaN,0.293340,NaN,0.026698,0.102251
005628,NaN,NaN,NaN,NaN,NaN,0.726834,0.238132,NaN,NaN,NaN


#### Uso de función: `topico_dominante`

Parámetro `df` es un DataFrame que contiene la distribución de probabilidad de tópicos en documentos. 

In [15]:
# Cada tópico es dominante en cuantos documentos?

dominante = topico_dominante(df=doctopics)
dominante.head()

,topico,docs
0,9,0.2299
1,2,0.1512
2,0,0.1112
3,4,0.1011
4,5,0.0885


### Distribución de probabilidad de palabras en cada tópico

#### Uso de función: `palabras_probables`

Genera un DataFrame con las palabras más probables en un tópico.

Parámetro `modelo` es un modelo de tópicos, `topico` es el número de tópico, y `n` indica cuantas palabras se quiere incuir en el resultado.

In [16]:
# Cuales son las palabras más probables en cada uno de los tópicos dominantes...

dfs = []
for topico in dominante['topico']:
    df = palabras_probables(modelo, topico, n=20)
    dfs.append(df)

palabras = pd.concat(dfs, ignore_index=True)

palabras.head()

,palabra,probabilidad,topico
0,crecimiento,0.037290,9
1,economía,0.013434,9
2,estados_unidos,0.011232,9
3,espera,0.010014,9
4,aumento,0.009473,9


## Estadísticas de corpus

### Estadísticas agregadas

In [17]:
# Estadísticas del corpus
stats = corpus.corpus_stats()
stats.tail()

,doc_id,archivo,fuente,frases,palabras
5623,005624,2005-12.txt,inflacion,1,7
5624,005625,2005-12.txt,inflacion,2,16
5625,005626,2005-12.txt,inflacion,3,46
5626,005627,2005-12.txt,inflacion,3,60
5627,005628,2005-12.txt,inflacion,2,26


In [18]:
# Estadísticas de tokens
tokens = corpus.corpus_tokens()
tokens.tail()

,doc_id,sent_id,tok_id,word,pos,ok_token,deterioro,mejora
173906,005628,2,12,esquema,NOUN,True,False,False
173907,005628,2,13,intervención,NOUN,True,False,False
173908,005628,2,14,discrecional,ADJ,True,False,False
173909,005628,2,15,mercado,NOUN,True,False,False
173910,005628,2,16,cambiario,ADJ,True,False,False


In [19]:
con_ngrams = corpus.corpus_ngramed()
con_ngrams.head(10)

,doc_id,sent_id,tok_id,word
0,000001,1,1,términos
1,000001,1,2,referencia
2,000001,1,3,decisiones
3,000001,1,4,política_monetaria
4,000001,1,5,junta_directiva_banco_república
5,000001,1,6,periódicamente
6,000001,1,7,análisis
7,000001,1,8,detallado
8,000001,1,9,resultados
9,000001,1,10,inflación


In [20]:
stats_frases = corpus.frases_stats()
stats_frases.head(10)

,doc_id,sent_id,ok_span,ok_token,deterioro,mejora
0,000001,1,True,18,0,0
1,000002,1,True,17,0,0
2,000003,1,True,17,0,0
3,000003,2,True,9,0,0
4,000004,1,True,13,0,0
5,000005,1,True,8,0,0
6,000005,2,True,7,0,0
7,000005,3,True,15,0,0
8,000005,4,True,10,0,0
9,000005,5,True,18,0,1
